In [1]:
#%pip install scikit-learn
#%pip install nltk
#%pip install utils
#%pip install emoji

In [6]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import sys
sys.path.append('../')
#from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player 
import emoji

#ACTION include in utils - but utils doesn't work for me here
def map_emoji_to_description(emoji_text): 
    emoji_description = emoji.demojize(emoji_text)
    return emoji_description


# Load Data

In [7]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [8]:
# Filter out the Spanish data and reindex
df_en = df[df["language"] == "en"]

#Reset index
df_en = df_en.reset_index(drop=True)
df_en

,data,player,language,publishedAt
0,{'content': 'Football\nFlorian Wirtz\'s goal f...,Exequiel Palacios,en,2023-02-16T23:56:00Z
1,{'content': '[1/4]\xa0Soccer Football - Europa...,Exequiel Palacios,en,2023-02-23T20:50:50Z
2,"{'content': ""By Will Pickworth For Mailonline\...",Exequiel Palacios,en,2023-02-23T20:53:59Z
3,{'content': '\nBUENOS AIRES (AP) — World Cup w...,Exequiel Palacios,en,2023-03-03T16:40:46Z
4,{'content': 'Sign In\nSign In\nThe Star Editio...,Exequiel Palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
297,{'content': 'Please refresh the page or naviga...,Moussa Diaby,en,2023-05-04T06:28:32Z
298,"{'content': ""By Oliver Salt For Mailonline\nPu...",Mykhaylo Mudryk,en,2023-05-05T14:58:51Z
299,{'content': 'We use cookies and other tracking...,Piero Hincapié,en,2023-04-27T04:57:02Z
300,{'content': 'Man City’s Alex Robertson makes d...,Piero Hincapié,en,2023-03-24T15:24:08Z


In [ ]:
print(df_en.loc[5, 'data'])

{'content': "World Cup winners Argentina will celebrate with their fans at two home games in March against Panama and Curacao in their first matches since their triumph in Qatar.\nArgentina coach Lionel Scaloni announced a 35-man squad on Friday which included Lionel Messi, Angel Di Maria and the other members of Argentina's winning squad in Qatar.\nManchester United forward Alejandro Garnacho is rewarded for impressing at Old Trafford with his first-ever callup. The 18-year-old, in his breakthrough season with Erik ten Hag's side, has contributed five goals and six assists in 30 matches this campaign.\nIn their first games since beating France in the World Cup final, Argentina will play Panama on March 23 at the Monumental Stadium in Buenos Aires. Five days later, they will host Curacao in another friendly at the northern city of Santiago del Estero.\nScaloni, 44, earlier this week extended his contract until the end of the 2026 World Cup.\nSquad:\nGoalkeepers: Emiliano Martinez (Asto

# Remove duplicates

In [ ]:
# Remove the similiar rows (The Function is imported from utils on top)

# Returned df is not working well

# Remove patterns

In [ ]:
#testing to find afftected rows
data_affected_row = df_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('new to', case=False)]

# Display the filtered rows
print(filtered_rows['data'].values)

In [10]:
# Create a copy
data_wo_pattern_en = df_en.copy()

#translate emojis into words
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group()), str(x)))

#remove patterns
patternlist_en = [
    r"^{\'content\': \'",
    r"^{\'content\'",
    r'http\S+',
    r"Copyright.*",
    r"Photo.*",
    r"All rights reserved.*",
    r'Published.*2023',
    r'Published by.*',
    r'Updated.*2023',
    r'Last updated.*2023',
    r'When you purchase through.*',
    #r'New to.*',
    r'Please refresh.*',
    r'From the section.*',
    r'We use cookies.*',
    r'Sign In\nSign In\nThe Star Edition\nCHANGE LOCATION\nThis copy is for your personal non-commercial use only. To order\n    presentation-ready copies of Toronto Star content for distribution\n    to colleagues, clients or customers, or inquire about\n    permissions/licensing, please go to: www.TorontoStarReprints.com\n'
]
#do we want to keep authors of the text? #data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r'By.*\n', "", str(x)))
#second version #data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r'Edited by.*\n', "", str(x)))


In [11]:
for pattern in patternlist_en:
    data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))
print(data_wo_pattern_en.loc[2, 'data'])
#it takes too long,if I would extend the pattern with + r'.*?\n' . But at the moment it deletes too much

: "By Will Pickworth For Mailonline\n\n\n5\nView  comments\n\nBayer Leverkusen, Shakhtar Donetsk, Union Berlin, Sevilla and Sporting Lisbon all joined Juventus,Manchester United\xa0and Roma\xa0in reaching the Europa League last-16.\nIt was a goal-heavy Thursday in the competition with 24 goals scored across the eight ties.\nShakhtar Donetsk continued their miraculous season as they overcame Rennes in the most dramatic fashion, beating them 5-4 on penalties.\nHaving led 2-1 from last week's first leg in Warsaw, a Karl Toko Ekambi strike gave the French side a 1-0 lead early on in the second half.\nThere were no more goals in the rest of normal time and the game headed into extra time with the score 2-2 on aggregate.\nShakhtar Donetsk reached the Europa League last-16 in miraculous fashion against Rennes\nKeeper Anatolii Trubin made three spectacular saves in the shootout to send them through\nBayer Leverkusen also reached the last-16 after beating AS Monaco 5-3 on penalties\nIbrahim Sal

In [ ]:
#check whether it worked
data_affected_row = data_wo_pattern_en.copy()
filtered_rows = data_affected_row[data_affected_row['data'].str.contains('Erik ten Hag is reportedly looking', case=False)]

# Display the filtered rows
print(filtered_rows)

Empty DataFrame
Columns: [data, player, language, publishedAt]
Index: []


In [12]:
data_wo_pattern_en['data'] = data_wo_pattern_en['data'].apply(lambda x: re.sub(r"Sign InnSign Inn*", "", str(x)))
print(data_wo_pattern_en.loc[5:15, 'data'])


5     : "World Cup winners Argentina will celebrate ...
6     Bayer Leverkusen became the third side to beat...
7     : "By Reuters\n\n\n7\nView  comments\n\nBayer ...
8     Bayer Leverkusen\'s Exequiel Palacios scored t...
9     by Tamojit Chakraborty March 19, 2023\nExequie...
10                                               : "\n.
11                                                     
12    : "Players from Barcelona and Real Madrid clas...
13    Craig Burley thinks Fulham players losing thei...
14    Argentina\'s World Cup winner Exequiel Palacio...
15    \nBarcelona took a huge step towards winning t...
Name: data, dtype: object
